# M12. Constants

A *constant* is a value that cannot be altered by a program during execution&mdash;it is specified in the code itself. Constants are ubiquitous. For example, consider a program to compute the wavelength of an electromagnetic wave in free space with frequency $f$:

In [4]:
def wavelength(f):
    c = 299792458 # speed of light in vacuum in m/s
    return c/f

wavelength(5000) # 5000 Hz

59958.4916

The value of $c$ does not change during runtime (hence the term *constant*&mdash;on second thought, this is a bit of a contrived example because $c$ doesn't change during the runtime of the universe, either).

In MIPS, it is possible to encode a constant into an assembly instruction as an *immediate operand*. The following is a table of MIPS-I ALU instructions that take an immediate:

| MNEMONIC |             DESCRIPTION             |
|:--------:|:-----------------------------------:|
| ADDI     | Add Immediate Word                  |
| ADDIU    | Add Immmediate Unsigned Word        |
| SLTI     | Set on Less Than Immediate          |
| SLTIU    | Set on Less Than Immediate Unsigned |
| ANDI     | And Immediate                       |
| ORI      | Or Immediate                        |
| XORI     | Exclusive-Or Immediate              |
| LUI      | Load Upper Immediate                |

We note that these instructions are still fixed to a size of 32-bits. Consequently, after budgeting for an opcode (6 bits) and two registers ($2\times 5 = 10$ bits) or an opcode, a register, and an *fmt* field ($6 + 5+5 = 16$ bits), immediates are limited to 16 bits. Signed immediates may therefore range from $-2^{15}$ to $2^{15}-1$.

Provided our constant is within this range, there is little to worry about. Suppose, for instance, that we want to load the value $174$ (8 bits) into register *s0*. We can perform

```
addi $s0, $zero, 174
```

How about values greater than $2^{15}-1 = 32767$? Attempting

```
addi $s0, $zero, 32768
```
results in an error:

```
Extended (pseudo) instruction or format not permitted.  See Settings.
```

(in general, you *should* have pseudo-instructions turned off. Uncheck *Settings* -> *Permit extended (pseudo) instructions and formats*). 

The workaround (and, in fact, the actual implementation beneath the pseudoinstruction *li*) is to divide the constant into its high-order 16 bits (2 bytes) and low-order 16 bits. This is more easily done if the constant is represented in hexadecimal: $1000000_{10} = 000F4240_{16}$.  

We may then do

```
lui $s0, 0x000F # Load upper two bytes of constant into upper two bytes of $s0
ori $s0, $s0, 0x4240 # Load lower two bytes of constant into lower two bytes of $s0
```
(This works because registers are initialized to 0&mdash;recall that x OR 0 = x).

Or, equivalently, one can do

```
lui $s0, 0x000F 
addi $s0, $s0, 0x4240 
```

I want to also mention how we load *static data* into registers. In MIPS, data fixed at compile-time are stored in the program's *data-segment*, starting at location 0x10010000.

For example, suppose we have

```
    .data
veryimportantnum: .word 68345
```

The pseudo-instruction *lw* allows us to load the value $68345$ by its label *veryimportantnum* (which is an address)

```
    .text
lw $s0, veryimportantnum
```

Strictly speaking, *lw* is not a pseudo-instruction but the usage above is permitted only if pseudo-instructions are permitted. Which they are not.

The more tedious implementation is 

```
    .text
lui $s0, 0x1001 # load the starting address of the data segment
lw $s0, 0($s0) # load the value at the starting address of the data segment (offset 0 for the first word) 

```

(unfortunately, this means that labels are not very useful when pseudo-instructions are turned off).